In [1]:
day = 16
year = 2022

# Part One

In [2]:
part = "a"


The sensors have led you to the origin of the distress signal: <br>yet another handheld device, just like the one the Elves gave you. <br>However, you don't see any Elves around; instead, <br>the device is surrounded by elephants! They must have gotten lost in these tunnels, and one of the elephants apparently figured out how to turn on the distress signal.

The ground rumbles again, much stronger this time. What kind of cave is this, exactly? You scan the cave with your handheld device; <br>it reports mostly igneous rock, some ash, pockets of pressurized gas, magma... this isn't just a cave, it's a volcano!

You need to get the elephants out of here, quickly. Your device estimates that you have 30 minutes before the volcano erupts,<br> so you don't have time to go back out the way you came in.

You scan the cave for other options and discover a network of pipes and pressure-release valves. <br>You aren't sure how such a system got into a volcano, <br>but you don't have time to complain; your device produces a report (your puzzle input) of each valve's flow rate if it were opened (in pressure per minute) and the tunnels you could use to move between the valves.

There's even a valve in the room you and the elephants are currently standing in labeled AA. <br>You estimate it will take you one minute to open a single valve and one minute to follow any tunnel from one valve to another. What is the most pressure you could release?

All of the valves begin closed. You start at valve AA, but it must be damaged or jammed or something: <br>its flow rate is 0, so there's no point in opening it. <br> However, you could spend one minute moving to valve BB and another minute opening it; doing so would release pressure during the remaining 28 minutes at a  <br>flow rate of 13, a total eventual pressure release of 28 * 13 = 364.  <br>Then, you could spend your third minute moving to valve CC and your fourth minute opening it, providing an additional 26 minutes of eventual pressure release at a flow rate of 2, or 52 total pressure released by valve CC.

Making your way through the tunnels like this, you could probably open many or all of the valves by the time 30 minutes have elapsed.  <br>However, you need to release as much pressure as possible, so you'll need to be methodical.

Work out the steps to release the most pressure in 30 minutes. What is the most pressure you can release?

In [3]:
import sys
import numpy as np
from copy import deepcopy

In [4]:
sys.path.append("..")

In [5]:
from utilities.utils import get_puzzle, submit_answer, read_sample_data, extract_numbers

In [6]:
test = read_sample_data(16)

In [7]:
def extract_valve(string: str):
    return string.split(" ")[1]

In [8]:
def extract_next_valves(string: str):
    valves = string.split(" to valve")[-1][1:].split(", ")
    valves = [v.strip() for v in valves]
    return valves

In [9]:
time_to_open_valve = 1
time_to_move_to_valve = 1
total_time = 30

In [10]:
class Valve:
    def __init__(self, valve_id: str, flow_rate: int, next_valves_ids: list):
        self.valve_id = valve_id
        self.flow_rate = flow_rate
        self.next_valves_ids = next_valves_ids
        self.next_valves = []
        self.open = False
        
    def add_next_valve(self, valve):
        if valve.valve_id in self.next_valves_ids:
            self.next_valves.append(valve)
        else:
            print(f"{valve.valve_id} is not accepted for this valve! Must be of {self.next_valves_ids}")
            
    def open_valve(self):
        self.open = True

In [11]:
routes = {}

In [12]:
def play_turn(current_valve, elapsed_time, open_valve: bool, move_valve: bool, route_id: int, route: list):
    route.append(current_valve.valve_id)
    print(f"{route_id} : elapsed time = {elapsed_time}")
    if elapsed_time == total_time:
        routes[route_id] = route
        print(f"Time up!")
        return current_valve, elapsed_time, route_id
    if not open_valve and not move_valve:
        raise ValueError("Must either open valve or move on!")
    # Option One - turn the current valve if closed
    if open_valve:
        if not current_valve.open:
            print(f"{route_id} : Opening valve of {current_valve.valve_id}")
            current_valve.open_valve()
            route.append(f"{current_valve.valve_id} opened")
            elapsed_time += time_to_open_valve
            current_valve, elapsed_time, route_id = play_turn(current_valve=current_valve, 
                                                             elapsed_time=elapsed_time, 
                                                             open_valve=False, 
                                                             move_valve=True, 
                                                             route_id=route_id, 
                                                             route=route)
        else:
            move_valve = True
         
    # Option Two - move to another valve
    if move_valve:
        for i, next_valve in enumerate(current_valve.next_valves):
            if next_valve.valve_id not in route:
                if next_valve.flow_rate == 0:
                    params = [(False, True)]
                else:
                    params = [(True, False), (False, True)]
                for j, param in enumerate(params):
                    print(f"{route_id} : Moving to {next_valve.valve_id} from {current_valve.valve_id}")
                    current_valve, elapsed_time, route_id = play_turn(current_valve=next_valve, 
                                                                      elapsed_time=elapsed_time+time_to_move_to_valve, 
                                                                     open_valve=param[0], 
                                                                     move_valve=param[1], 
                                                                     route_id=route_id+i+j, 
                                                                     route=route)
                
    if elapsed_time == total_time:
        routes[route_id] = route
        print(f"Time up!")
    return current_valve, elapsed_time, route_id
    

In [13]:
test

['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
 'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
 'Valve CC has flow rate=2; tunnels lead to valves DD, BB',
 'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
 'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
 'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
 'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
 'Valve HH has flow rate=22; tunnel leads to valve GG',
 'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
 'Valve JJ has flow rate=21; tunnel leads to valve II']

In [14]:
valves = []
for row in test:
    valve_id = extract_valve(row)
    flow_rate = extract_numbers(row)
    next_valves = extract_next_valves(row)
    valve = Valve(valve_id, flow_rate, next_valves)
    valves.append(valve)

In [15]:
for valve in valves:
    for valve_id in valve.next_valves_ids:
        for v in valves:
            if v.valve_id == valve_id:
                valve.add_next_valve(v)

In [16]:
# play_turn(valves[0], 0, False, True, 0, [])

## Breadth first Search using Linked List 

In [17]:
processed = [False] * len(valves)
discovered = [False] * len(valves)

In [21]:
flow_rates = {}
for valve in valves:
    flow_rates[valve.valve_id] = valve.flow_rate[0]
flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

In [18]:

adj_list = {}
mylist = []
def add_node(node):
  if node not in mylist:
    mylist.append(node)
  else:
    print("Node ",node," already exists!")
 
def add_edge(node1, node2):
  temp = []
  if node1 in mylist and node2 in mylist:
    if node1 not in adj_list:
      temp.append(node2)
      adj_list[node1] = temp
   
    elif node1 in adj_list:
      temp.extend(adj_list[node1])
      temp.append(node2)
      adj_list[node1] = temp
       
  else:
    print("Nodes don't exist!")
 
def graph():
  for node in adj_list:
    print(node, " ---> ", [i for i in adj_list[node]])
 
#Adding nodes
for node in valves:
    add_node(node.valve_id)
    
for node in valves:
    for nn in node.next_valves_ids:
        add_edge(node.valve_id, nn)
    
    
# add_node(0)
# add_node(1)
# add_node(2)
# add_node(3)
# add_node(4)
# #Adding edges
# add_edge(0,1)
# add_edge(1,2)
# add_edge(2,3)
# add_edge(3,0)
# add_edge(3,4)
# add_edge(4,0)
 
#Printing the graph
graph()
 
#Printing the adjacency list
print(adj_list)

AA  --->  ['DD', 'II', 'BB']
BB  --->  ['CC', 'AA']
CC  --->  ['DD', 'BB']
DD  --->  ['CC', 'AA', 'EE']
EE  --->  ['FF', 'DD']
FF  --->  ['EE', 'GG']
GG  --->  ['FF', 'HH']
HH  --->  ['GG']
II  --->  ['AA', 'JJ']
JJ  --->  ['II']
{'AA': ['DD', 'II', 'BB'], 'BB': ['CC', 'AA'], 'CC': ['DD', 'BB'], 'DD': ['CC', 'AA', 'EE'], 'EE': ['FF', 'DD'], 'FF': ['EE', 'GG'], 'GG': ['FF', 'HH'], 'HH': ['GG'], 'II': ['AA', 'JJ'], 'JJ': ['II']}


In [26]:
def bfs(visit_complete, graph, current_node):
    queue = []
    elapsed_time = 0
    route_id = 0
    route_ids = [0]
    route = []
    # route.append(current_node)
    queue.append((current_node, elapsed_time, route_id, route))
    routes = {}
    node_states = {}
    for n in graph.keys():
        node_states[n] = dict(flow=flow_rates[n], 
                             open=False)

 
    while queue:
        # print(queue)
        s, elapsed_time, route_id, route = queue.pop(0)
        if not node_states[s]["open"] and node_states[s]["flow"] > 0:
            node_states[s]["open"] = True
            elapsed_time += 1
        if route_id in routes.keys():
            route_id = max(route_ids) + 1
        if route_id not in route_ids:
            route_ids.append(route_id)
        # print(f"{s} added to {route_id} : {route}")
        route.append(s)
        if elapsed_time >= 30:
            print(f"Route {route_id} complete!")
            routes[route_id] = route
        else:
            # print(f"Node {s} : elapsed time -> {elapsed_time} : route -> {route_id}")

            # todo option to open the valve
            neighbours = [n for n in graph[s]]
            # print(f"Neighbours of {s} -> {neighbours}")
            for i, neighbour in enumerate(neighbours):
                queue.append((neighbour, elapsed_time +2, route_id+i, deepcopy(route)))  
    return routes


In [25]:
len(routes)

5965

In [ ]:
routes = bfs([], adj_list, 'AA')

Route 0 complete!
Route 1 complete!
Route 2 complete!
Route 21 complete!
Route 22 complete!
Route 23 complete!
Route 24 complete!
Route 3 complete!
Route 25 complete!
Route 26 complete!
Route 27 complete!
Route 4 complete!
Route 28 complete!
Route 29 complete!
Route 30 complete!
Route 31 complete!
Route 5 complete!
Route 32 complete!
Route 33 complete!
Route 34 complete!
Route 35 complete!
Route 36 complete!
Route 37 complete!
Route 38 complete!
Route 39 complete!
Route 40 complete!
Route 41 complete!
Route 42 complete!
Route 43 complete!
Route 44 complete!
Route 45 complete!
Route 46 complete!
Route 47 complete!
Route 48 complete!
Route 49 complete!
Route 50 complete!
Route 51 complete!
Route 52 complete!
Route 53 complete!
Route 54 complete!
Route 55 complete!
Route 56 complete!
Route 57 complete!
Route 58 complete!
Route 59 complete!
Route 6 complete!
Route 60 complete!
Route 61 complete!
Route 62 complete!
Route 63 complete!
Route 64 complete!
Route 65 complete!
Route 66 complete!


In [49]:
routes

{0: ['AA', 'DD', 'CC', 'DD', 'CC', 'DD', 'CC'],
 1: ['AA', 'DD', 'CC', 'DD', 'CC', 'DD', 'AA'],
 2: ['AA', 'DD', 'CC', 'DD', 'CC', 'DD', 'EE'],
 9: ['AA', 'DD', 'CC', 'DD', 'CC', 'BB', 'CC'],
 10: ['AA', 'DD', 'CC', 'DD', 'CC', 'BB', 'AA'],
 11: ['AA', 'DD', 'CC', 'DD', 'AA', 'DD', 'CC'],
 12: ['AA', 'DD', 'CC', 'DD', 'AA', 'DD', 'AA'],
 3: ['AA', 'DD', 'CC', 'DD', 'AA', 'DD', 'EE'],
 13: ['AA', 'DD', 'CC', 'DD', 'AA', 'II', 'AA'],
 14: ['AA', 'DD', 'CC', 'DD', 'AA', 'II', 'JJ'],
 15: ['AA', 'DD', 'CC', 'DD', 'AA', 'BB', 'CC'],
 4: ['AA', 'DD', 'CC', 'DD', 'AA', 'BB', 'AA'],
 16: ['AA', 'DD', 'CC', 'DD', 'EE', 'FF', 'EE'],
 17: ['AA', 'DD', 'CC', 'DD', 'EE', 'FF', 'GG'],
 18: ['AA', 'DD', 'CC', 'DD', 'EE', 'DD', 'CC'],
 19: ['AA', 'DD', 'CC', 'DD', 'EE', 'DD', 'AA'],
 5: ['AA', 'DD', 'CC', 'DD', 'EE', 'DD', 'EE'],
 20: ['AA', 'DD', 'CC', 'BB', 'CC', 'DD', 'CC'],
 21: ['AA', 'DD', 'CC', 'BB', 'CC', 'DD', 'AA'],
 22: ['AA', 'DD', 'CC', 'BB', 'CC', 'DD', 'EE'],
 23: ['AA', 'DD', 'CC', 'BB